In [1]:
import os
import pandas as pd
import numpy as np
import ee

In [2]:
# 认证并初始化Google Earth Engine
ee.Authenticate()  # 只需要在第一次运行时执行
ee.Initialize(project= 'ee-zongrong-flood')

In [3]:
# Load the image for 2018 and select the built_height band
image_2018 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_H/2018').select('built_height')

# Calculate the min and max built_height values for the 2018 image
min_max = image_2018.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee.Geometry.BBox(-180, -90, 180, 90),
    scale=30,
    bestEffort=True
).getInfo()

min_value = min_max['built_height_min']
max_value = min_max['built_height_max']

print(f" Min value: {min_value}, Max value: {max_value}")

 Min value: 0, Max value: 47.738197326660156


In [7]:
# Function to get built height value at given coordinates and normalize it using logarithmic transformation
def get_built_height(coords):
    point = ee.Geometry.Point(coords)
    built_value = image_2018.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    ).get('built_height')
    
    value = built_value.getInfo()
    if value is None:
        # If the value is None, use nearest neighbor interpolation to fill the gap
        built_value = image_2018.reduceNeighborhood(
            reducer=ee.Reducer.mean(),
            kernel=ee.Kernel.square(5),  # Increase the kernel size
            inputWeight=True,
            optimization='boxcar'
        ).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30,
            bestEffort=True
        ).get('built_height')
        
        value = built_value.getInfo()
        if value is None:
            return 0
    
    
    normalized_value = (value - min_value) / (max_value - min_value) * 10
    print(f"Coordinates: {coords}, Original value: {value}, Normalized value: {normalized_value}")  # Debugging line
    return normalized_value

# Specify the input and output directories
input_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/Sampling'
output_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/building_height'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each CSV file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_dir, filename)
        df = pd.read_csv(input_csv)
        
        # Create a new column for built height values for 2018
        df['Built_Height_2018'] = df.apply(lambda row: get_built_height([row['longitude'], row['latitude']]), axis=1)
        
        # Define the output filename
        output_csv = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_building_height.csv")
        
        # Save the new dataframe to a CSV file
        df.to_csv(output_csv, index=False)
        
        print(f"Processed {input_csv} and saved with built height values as {output_csv}")

print("All CSV files processed.")

Coordinates: [114.276307, 22.2695341], Original value: 0, Normalized value: 0.0
Coordinates: [114.1023043902492, 22.37115357347453], Original value: 14.194741249084473, Normalized value: 2.973455648514234
Coordinates: [114.23624955805002, 22.287083445560008], Original value: 0, Normalized value: 0.0
Coordinates: [114.2983256, 22.2618995], Original value: 0, Normalized value: 0.0
Coordinates: [114.18624955824998, 22.34541677866001], Original value: 7.707016944885254, Normalized value: 1.614434012274935
Coordinates: [114.1940713503955, 22.29561213573655], Original value: 0, Normalized value: 0.0
Coordinates: [114.1706041480453, 22.26166046887501], Original value: 2.503497838973999, Normalized value: 0.5244223659815241
Coordinates: [114.25291622465, 22.30375011216001], Original value: 27.743396759033203, Normalized value: 5.81157193037523
Coordinates: [114.21958289145, 22.32041677876001], Original value: 35.03692626953125, Normalized value: 7.339390306211735
Coordinates: [114.16958289165,

Coordinates: [114.2868454941899, 22.24583502676082], Original value: 0, Normalized value: 0.0
Coordinates: [114.18624955824998, 22.31208344546001], Original value: 27.57699203491211, Normalized value: 5.776714157472238
Coordinates: [114.2041615919109, 22.31957680366856], Original value: 7.729004383087158, Normalized value: 1.6190398498291791
Coordinates: [114.3082692, 22.3633873], Original value: 0, Normalized value: 0.0
Coordinates: [114.2240295, 22.36166], Original value: 0, Normalized value: 0.0
Coordinates: [114.275856, 22.2958202], Original value: 4.4137187004089355, Normalized value: 0.9245675261273479
Coordinates: [114.20291622485, 22.287083445560008], Original value: 25.003652572631836, Normalized value: 5.237661657296839
Coordinates: [114.234072, 22.3646607], Original value: 0, Normalized value: 0.0
Coordinates: [114.16124955835, 22.26208344566001], Original value: 0, Normalized value: 0.0
Coordinates: [114.12791622515, 22.32041677876001], Original value: 18.051761627197266, N

Coordinates: [114.2603037001417, 22.34626232578898], Original value: 2.5, Normalized value: 0.523689653149897
Coordinates: [114.26958289125002, 22.29541677886001], Original value: 8.286890983581543, Normalized value: 1.735903625953131
Coordinates: [114.10291622525, 22.34541677866001], Original value: 0, Normalized value: 0.0
Coordinates: [114.2941447628782, 22.3616944664604], Original value: 0, Normalized value: 0.0
Coordinates: [114.11124955855, 22.370416778560006], Original value: 23.04759407043457, Normalized value: 4.827914617874201
Coordinates: [114.1997153221382, 22.36128378949011], Original value: 0, Normalized value: 0.0
Coordinates: [114.2603378, 22.2958908], Original value: 0, Normalized value: 0.0
Coordinates: [114.22791622475, 22.278750112260006], Original value: 29.489545822143555, Normalized value: 6.177348009258542
Coordinates: [114.3023995, 22.279642], Original value: 0, Normalized value: 0.0
Coordinates: [114.22791622475, 22.26208344566001], Original value: 0, Normaliz

Coordinates: [114.3012422, 22.3210019], Original value: 0, Normalized value: 0.0
Coordinates: [114.2540134982365, 22.26213287085497], Original value: 2.5, Normalized value: 0.523689653149897
Coordinates: [114.15291622505003, 22.278750112260006], Original value: 41.368289947509766, Normalized value: 8.665658165606304
Coordinates: [114.24458289135003, 22.328750112060007], Original value: 2.5, Normalized value: 0.523689653149897
Coordinates: [114.22791622475, 22.32041677876001], Original value: 32.56948471069336, Normalized value: 6.822520860565552
Coordinates: [114.23624955805002, 22.32041677876001], Original value: 20.53287696838379, Normalized value: 4.3011420870969665
Coordinates: [114.0783606333855, 22.35243342008602], Original value: 2.828683376312256, Normalized value: 0.592540886484738
Coordinates: [114.3112488, 22.3428001], Original value: 0, Normalized value: 0.0
Coordinates: [114.17791622495002, 22.328750112060007], Original value: 13.730443000793457, Normalized value: 2.876196

Coordinates: [114.17791622495002, 22.287083445560008], Original value: 0, Normalized value: 0.0
Coordinates: [114.1444611301621, 22.27154068196903], Original value: 0, Normalized value: 0.0
Coordinates: [114.16124955835, 22.33708344536001], Original value: 34.00782775878906, Normalized value: 7.123819009352674
Coordinates: [114.2853940546976, 22.27817094982018], Original value: 0, Normalized value: 0.0
Coordinates: [114.22791622475, 22.30375011216001], Original value: 19.396839141845703, Normalized value: 4.0631695849590095
Coordinates: [114.1867881386808, 22.26271600889114], Original value: 11.953595161437988, Normalized value: 2.503989641595099
Coordinates: [114.2189000174403, 22.33638468578732], Original value: 3.9927845001220703, Normalized value: 0.8363919719884848
Coordinates: [114.2029657281529, 22.35434426456857], Original value: 0, Normalized value: 0.0
Coordinates: [114.21124955815004, 22.27041677896001], Original value: 0, Normalized value: 0.0
Coordinates: [114.0693344, 22.

...

Coordinates: [139.45291612385, 35.52041672596001], Original value: 20.298410415649414, Normalized value: 4.25202700402628
Coordinates: [140.2611259723665, 35.45457538905087], Original value: 0, Normalized value: 0.0
Coordinates: [140.0773517973294, 35.45378106001398], Original value: 2.7399466037750244, Normalized value: 0.5739526746320723
Coordinates: [139.66124945635, 35.82875005806001], Original value: 15.429678916931152, Normalized value: 3.2321452800887815
Coordinates: [139.71124945615003, 35.56208339246001], Original value: 20.36432647705078, Normalized value: 4.265834827759195
Coordinates: [139.59458278994998, 35.57041672576001], Original value: 14.154427528381348, Normalized value: 2.9650108971493534
Coordinates: [140.06124945475, 35.64541672546001], Original value: 18.239713668823242, Normalized value: 3.820779729911792
Coordinates: [139.40291612405002, 35.67875005866001], Original value: 11.33552360534668, Normalized value: 2.3745185700625893
Coordinates: [140.33624945365, 35

Coordinates: [139.59458278994998, 35.82041672476001], Original value: 11.837960243225098, Normalized value: 2.479766917510729
Coordinates: [139.80291612245003, 35.67041672536001], Original value: 25.00146484375, Normalized value: 5.237203380905113
Coordinates: [139.65291612305003, 35.62875005886001], Original value: 12.961074829101562, Normalized value: 2.7150323126808233
Coordinates: [140.14458278774998, 35.66208339206001], Original value: 12.124377250671387, Normalized value: 2.5397643668250405
Coordinates: [140.29458278715003, 35.72041672516001], Original value: 2.5, Normalized value: 0.523689653149897
Coordinates: [139.7385218920132, 35.51284518167103], Original value: 14.661791801452637, Normalized value: 3.071291465223494
Coordinates: [139.68624945624998, 35.57875005906001], Original value: 12.450164794921875, Normalized value: 2.6080089932446784
Coordinates: [140.2678575210163, 35.57008375974329], Original value: 2.618356227874756, Normalized value: 0.5484824259194414
Coordinate

Coordinates: [139.62791612315, 35.66208339206001], Original value: 14.291725158691406, Normalized value: 2.993771436507504
Coordinates: [139.5785985121674, 35.83729266272209], Original value: 12.692197799682617, Normalized value: 2.6587090653702705
Coordinates: [139.65291612305003, 35.52875005926001], Original value: 16.667783737182617, Normalized value: 3.491498353641064
Coordinates: [139.44458279055, 35.66208339206001], Original value: 11.255356788635254, Normalized value: 2.357725557087494
Coordinates: [139.71958278945, 35.57875005906001], Original value: 12.306459426879883, Normalized value: 2.5779061875064024
Coordinates: [140.23624945404998, 35.66208339206001], Original value: 0, Normalized value: 0.0
Coordinates: [140.12791612115, 35.47875005946001], Original value: 5.344106197357178, Normalized value: 1.1194612483560782
Coordinates: [139.46124945715002, 35.73708339176001], Original value: 10.48662281036377, Normalized value: 2.1966943449092806
Coordinates: [140.26124945395003, 

Coordinates: [139.76958278925002, 35.84541672466001], Original value: 11.417022705078125, Normalized value: 2.391590664170745
Coordinates: [139.61958278985003, 35.50375005936001], Original value: 15.093079566955566, Normalized value: 3.1616358413531036
Coordinates: [139.62791612315, 35.853750057960006], Original value: 13.252045631408691, Normalized value: 2.7759836720956104
Coordinates: [139.41124945735004, 35.49541672606001], Original value: 10.699383735656738, Normalized value: 2.241262622977491
Coordinates: [139.37791612415003, 35.61208339226001], Original value: 23.380327224731445, Normalized value: 4.897614181940282
Coordinates: [139.70291612285, 35.83708339136001], Original value: 14.488951683044434, Normalized value: 3.0350856325596625
Coordinates: [139.34458279095003, 35.88708339116001], Original value: 8.128127098083496, Normalized value: 1.7026464243014503
Coordinates: [139.56958279005, 35.80375005816001], Original value: 13.537002563476562, Normalized value: 2.8356752708625

Coordinates: [139.48624945705, 35.77041672496001], Original value: 12.632954597473145, Normalized value: 2.6462990445636434
Coordinates: [140.29458278715003, 35.71208339186001], Original value: 7.649620532989502, Normalized value: 1.6024108494598412
Coordinates: [140.20291612085003, 35.88708339116001], Original value: 2.739047050476074, Normalized value: 0.5737642399300256
Coordinates: [140.0519799921092, 35.8792328858045], Original value: 8.815689086914062, Normalized value: 1.846674064081343
Coordinates: [140.21958278745, 35.89541672446001], Original value: 8.485236167907715, Normalized value: 1.777452154266621
Coordinates: [140.1214722498299, 35.78692459544871], Original value: 3.388747215270996, Normalized value: 0.709860741511179
Coordinates: [139.3768156094037, 35.47017254165949], Original value: 0, Normalized value: 0.0
Coordinates: [140.31124945375, 35.55375005916001], Original value: 9.878687858581543, Normalized value: 2.069346647294667
Coordinates: [139.41124945735004, 35.80